In [28]:
import pandas as pd
import numpy as np

In [29]:
df = pd.read_csv('event_data.csv')

In [30]:
df.describe()

,purchase_amount
count,6207.000000
mean,30.035444
std,14.101013
min,10.000000
25%,20.000000
50%,30.000000
75%,40.000000
max,50.000000


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79742 entries, 0 to 79741
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          79742 non-null  object 
 1   event_date       79742 non-null  object 
 2   event_type       79742 non-null  object 
 3   purchase_amount  6207 non-null   float64
dtypes: float64(1), object(3)
memory usage: 2.4+ MB


In [32]:
df.dtypes

user_id             object
event_date          object
event_type          object
purchase_amount    float64
dtype: object

Converting 'event_date' to `datetime`. We will add a new column 'event_date_datetime' to not overwrite the original columns.

Since the events are all in 2019, there is no need to distinguish years afterwards in the 'week_number' column.

In [33]:
df['event_date_datetime'] = pd.to_datetime(df['event_date'])
df.dtypes

user_id                        object
event_date                     object
event_type                     object
purchase_amount               float64
event_date_datetime    datetime64[ns]
dtype: object

First, lets check if all events are happening in the same year.

In [34]:
df['event_date_datetime'].min()

Timestamp('2019-07-29 00:02:15')

In [35]:
df['event_date_datetime'].max()

Timestamp('2019-09-01 23:59:54')

### 2.Highlight user cohorts based on the week of registration in the application. The cohort identifier should be the week ordinal (for example, the week from July 29 to August 4 should have identifier 31).

Creating 'week_number' column using *dt.strftime('%U')*, then converting it to *int* dtype.

In [45]:
df['week_number_cohort'] = df['event_date_datetime'].dt.strftime('%U').astype(int)
df

,user_id,event_date,event_type,purchase_amount,event_date_datetime,week_number_cohort
0,c40e6a,2019-07-29 00:02:15,registration,NaN,2019-07-29 00:02:15,30
1,a2b682,2019-07-29 00:04:46,registration,NaN,2019-07-29 00:04:46,30
2,9ac888,2019-07-29 00:13:22,registration,NaN,2019-07-29 00:13:22,30
3,93ff22,2019-07-29 00:16:47,registration,NaN,2019-07-29 00:16:47,30
4,65ef85,2019-07-29 00:19:23,registration,NaN,2019-07-29 00:19:23,30
...,...,...,...,...,...,...
79737,930c23,2019-09-01 23:57:41,simple_event,NaN,2019-09-01 23:57:41,35
79738,a84999,2019-09-01 23:57:50,simple_event,NaN,2019-09-01 23:57:50,35
79739,175e4d,2019-09-01 23:59:40,simple_event,NaN,2019-09-01 23:59:40,35
79740,1c2210,2019-09-01 23:59:51,simple_event,NaN,2019-09-01 23:59:51,35


### 3. How many unique users in the cohort with ID 33?

We need to first .query() or filter the dataframe to 'week_number_cohort'==33. Then, we calculate the count of unique user_id using np.nunique().

In [46]:
unique_users_id_33 = df.query('week_number_cohort==33')['user_id'].nunique()

print(f'There are {unique_users_id_33} in the ID 33 week cohort')

There are 5776 in the ID 33 week cohort


### 4. For each event, highlight the indicator lifetime - the weekly lifetime of the cohort. The lifetime indicator is calculated based on the serial number of the week in which the event is committed, relative to the week of registration. For example, an event committed on August 3 by a user from a cohort of registrants at 31 weeks will be committed on the zero week of lifetime, and an event committed by the same user on August 5 will be committed on the first week of lifetime.

Given that the first/minimal week time cohort will be given when the user df['event_type'] = 'registration', from there we can calculate how many weeks past the registration the event happened.
To do so, we groupby the user_id and calculate based on the df['week_number_cohort'] column.
.transform() will allow us to calculate the delta between the first week of registration and the event week.

In [49]:
df['indicator_lifetime'] = df.groupby('user_id')['week_number_cohort'].transform(lambda x: x - x.min())


In [50]:
df

,user_id,event_date,event_type,purchase_amount,event_date_datetime,week_number_cohort,indicator_lifetime
0,c40e6a,2019-07-29 00:02:15,registration,NaN,2019-07-29 00:02:15,30,0
1,a2b682,2019-07-29 00:04:46,registration,NaN,2019-07-29 00:04:46,30,0
2,9ac888,2019-07-29 00:13:22,registration,NaN,2019-07-29 00:13:22,30,0
3,93ff22,2019-07-29 00:16:47,registration,NaN,2019-07-29 00:16:47,30,0
4,65ef85,2019-07-29 00:19:23,registration,NaN,2019-07-29 00:19:23,30,0
...,...,...,...,...,...,...,...
79737,930c23,2019-09-01 23:57:41,simple_event,NaN,2019-09-01 23:57:41,35,4
79738,a84999,2019-09-01 23:57:50,simple_event,NaN,2019-09-01 23:57:50,35,3
79739,175e4d,2019-09-01 23:59:40,simple_event,NaN,2019-09-01 23:59:40,35,4
79740,1c2210,2019-09-01 23:59:51,simple_event,NaN,2019-09-01 23:59:51,35,3


### 4. Build a summary table of changes in the Retention Rate for cohorts depending on lifetime.

Let's calculate it by grouping both the 'week_number_cohort' and the 'indicator_lifetime', and then counting the unique 'user_id' in that grouping.

In [59]:
summary = pd.DataFrame(df.groupby(['week_number_cohort','indicator_lifetime'])['user_id'].nunique())
summary

user_id
week_number_cohort indicator_lifetime         
30                 0                      1675
31                 0                      1997
                   1                      1576
32                 0                      2003
                   1                      1853
                   2                      1087
33                 0                      1985
                   1                      1858
                   2                      1316
                   3                       617
34                 0                      2043
                   1                      1848
                   2                      1327
                   3                       718
                   4                       257
35                 0                       293
                   1                      1017
                   2                       707
                   3                       451
                   4                       199
                   5                        36

### 6. What is the 3 week retention rate for a cohort with ID 32? Give the answer in percent, rounded to 2 decimal places, inclusive.

Unstacking the summary table to better understand the usage.

In [61]:
uns_summary = summary.unstack()
uns_summary

user_id                                    
indicator_lifetime       0       1       2      3      4     5
week_number_cohort                                            
30                  1675.0     NaN     NaN    NaN    NaN   NaN
31                  1997.0  1576.0     NaN    NaN    NaN   NaN
32                  2003.0  1853.0  1087.0    NaN    NaN   NaN
33                  1985.0  1858.0  1316.0  617.0    NaN   NaN
34                  2043.0  1848.0  1327.0  718.0  257.0   NaN
35                   293.0  1017.0   707.0  451.0  199.0  36.0

The 3 week retention is given by diving the usage of users that first registered in the week 32 (2003) and still used the app three weeks later, where the 'indicator_lifetime' is 3 (617).

In [67]:
print(f'The three week retention rate for the cohort 32 is {round(617/2003,2)}')

The three week retention rate for the cohort 32 is 0.31


### 7. Build a summary table of changes in the indicator ARPPU (Average Revenue Per Paying User) for cohorts depending on lifetime.

To do so, let's group by the dataframe by the 'week_number_cohort', the 'lifetime_indicator' and sum the purchase ammount to get each week lifetime indicator revenue. Then, we compare it against paying users by filtering out unique users that did not make a purchase *(purchase_amount>0)*

In [121]:
# Calculate the ARPPU for each cohort and lifetime week
lifetime_rev_week = df.groupby(['week_number_cohort', 'indicator_lifetime'])['purchase_amount'].sum()
lifetime_rev_week


week_number_cohort  indicator_lifetime
30                  0                      6010.0
31                  0                     10970.0
                    1                     17430.0
32                  0                      9380.0
                    1                     21670.0
                    2                     10910.0
33                  0                     10350.0
                    1                     20410.0
                    2                     13310.0
                    3                      5650.0
34                  0                     11260.0
                    1                     19250.0
                    2                     13370.0
                    3                      6270.0
                    4                      1840.0
35                  0                       310.0
                    1                      3310.0
                    2                      2010.0
                    3                      1840.0
           

In [122]:
uns_lifetime_rev_week = lifetime_rev_week.unstack()
uns_lifetime_rev_week

indicator_lifetime,0,1,2,3,4,5
week_number_cohort,,,,,,
30,6010.0,NaN,NaN,NaN,NaN,NaN
31,10970.0,17430.0,NaN,NaN,NaN,NaN
32,9380.0,21670.0,10910.0,NaN,NaN,NaN
33,10350.0,20410.0,13310.0,5650.0,NaN,NaN
34,11260.0,19250.0,13370.0,6270.0,1840.0,NaN
35,310.0,3310.0,2010.0,1840.0,760.0,120.0


In [124]:
paying_users_lifetime = df.query('purchase_amount>0').groupby(['week_number_cohort', 'indicator_lifetime'])['user_id'].nunique()
paying_users_lifetime

week_number_cohort  indicator_lifetime
30                  0                     200
31                  0                     326
                    1                     505
32                  0                     289
                    1                     611
                    2                     305
33                  0                     301
                    1                     586
                    2                     368
                    3                     164
34                  0                     334
                    1                     543
                    2                     382
                    3                     191
                    4                      59
35                  0                       8
                    1                     108
                    2                      59
                    3                      58
                    4                      23
                    5                    

In [125]:
uns_paying_users_lifetime = paying_users_lifetime.unstack()
uns_paying_users_lifetime

indicator_lifetime,0,1,2,3,4,5
week_number_cohort,,,,,,
30,200.0,NaN,NaN,NaN,NaN,NaN
31,326.0,505.0,NaN,NaN,NaN,NaN
32,289.0,611.0,305.0,NaN,NaN,NaN
33,301.0,586.0,368.0,164.0,NaN,NaN
34,334.0,543.0,382.0,191.0,59.0,NaN
35,8.0,108.0,59.0,58.0,23.0,3.0


To divide tables both with two indexes, do not call pd.DataFrame beforehand - instead leave them as tables and then call it afterwards

In [131]:
arrpu = pd.DataFrame(lifetime_rev_week.div(paying_users_lifetime,axis = 'index').rename('arrpu'))
arrpu

arrpu
week_number_cohort indicator_lifetime           
30                 0                   30.050000
31                 0                   33.650307
                   1                   34.514851
32                 0                   32.456747
                   1                   35.466448
                   2                   35.770492
33                 0                   34.385382
                   1                   34.829352
                   2                   36.168478
                   3                   34.451220
34                 0                   33.712575
                   1                   35.451197
                   2                   35.000000
                   3                   32.827225
                   4                   31.186441
35                 0                   38.750000
                   1                   30.648148
                   2                   34.067797
                   3                   31.724138
                   4                   33.043478
                   5                   40.000000

In [133]:
uns_arrpu = arrpu.unstack()
uns_arrpu

arrpu                                              \
indicator_lifetime          0          1          2          3          4   
week_number_cohort                                                          
30                  30.050000        NaN        NaN        NaN        NaN   
31                  33.650307  34.514851        NaN        NaN        NaN   
32                  32.456747  35.466448  35.770492        NaN        NaN   
33                  34.385382  34.829352  36.168478  34.451220        NaN   
34                  33.712575  35.451197  35.000000  32.827225  31.186441   
35                  38.750000  30.648148  34.067797  31.724138  33.043478   

                          
indicator_lifetime     5  
week_number_cohort        
30                   NaN  
31                   NaN  
32                   NaN  
33                   NaN  
34                   NaN  
35                  40.0

### 8. What is the 3-week ARPPU of a cohort with ID 31? Give the answer with a floating point number, rounded to 2 decimal places, inclusive.

Using the summarized ARRPU table, we find that the 3 week arrpu is given in the intersectionof the 2 indicator lifetime in the week_number_cohort **32**, when it has passed 3 weeks from the first purchase made in the cohort 31

In [134]:
print(f'The three week ARRPU for the cohort 31 is {round(35.770492,2)}')

The three week retention rate for the cohort 31 is 35.77


### 9. What is the median time between user registration and first purchase? Give the answer in seconds (!) As an integer.

First we need to get the user and the fist(minimal) date that he made a purchase. Then, we evaluate it against his registration date.

In [98]:
df['event_type'].unique()

array(['registration', 'simple_event', 'purchase'], dtype=object)

In [105]:
registration = df.query('event_type=="registration"').groupby('user_id')['event_date_datetime'].min()
registration

user_id
00049f   2019-08-18 16:21:02
0005f9   2019-08-29 16:24:42
00082c   2019-08-03 17:37:37
000ff1   2019-08-27 11:22:00
001d5d   2019-08-23 13:24:11
                 ...        
ffe1a7   2019-08-05 03:05:13
ffe60a   2019-08-26 06:35:05
fff039   2019-08-26 21:46:49
fff22e   2019-08-16 08:23:33
fff4aa   2019-08-08 23:49:26
Name: event_date_datetime, Length: 9996, dtype: datetime64[ns]

In [101]:
first_purchase = df.query('purchase_amount>0').groupby('user_id')['event_date_datetime'].min()
first_purchase

user_id
00049f   2019-08-18 17:31:56
00082c   2019-08-14 12:57:41
001d5d   2019-08-26 13:03:22
002195   2019-08-18 22:35:43
0025c1   2019-08-10 11:37:00
                 ...        
ffb3f3   2019-08-04 06:47:40
ffbe61   2019-08-21 21:14:07
ffc66a   2019-08-16 06:05:05
ffc88b   2019-08-09 15:36:48
fff4aa   2019-08-23 08:26:15
Name: event_date_datetime, Length: 4283, dtype: datetime64[ns]

dt.total_seconds() function convert a datetime to seconds.

https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.total_seconds.html

In [109]:
delta_fist_purchase = first_purchase - registration
total_secs = delta_fist_purchase.dt.total_seconds()

In [110]:
total_secs

user_id
00049f       4254.0
0005f9          NaN
00082c     933604.0
000ff1          NaN
001d5d     257951.0
            ...    
ffe1a7          NaN
ffe60a          NaN
fff039          NaN
fff22e          NaN
fff4aa    1240609.0
Name: event_date_datetime, Length: 9996, dtype: float64

In [111]:
median_first_purchase = int(total_secs.median())

In [112]:
print(f'The median between user register and the first purchase is {median_first_purchase} seconds. ')

The median between user register and the first purchase is 434774 seconds
